This dataset generation code is written using the hints given in the project scope document, and the proof from the reference paper attached along.

In [ ]:
import numpy as np
import pandas as pd
import itertools
from scipy.optimize import linprog

'''
This function is used to compute m-height and generate the dataset
The code here just checks if it is working fine for each combination of (n, k, m)
As this optimization takes GPU operations, to generate data for 1M, a modified version of this code was run on HPRC
'''

def m_height_linprog_exact(P, n, k, m):

    # Constructing generator matrix G = [I_k | P]
    G = np.hstack((np.eye(k), P))
    indices = list(range(n))

    # All the possible psi vectors of length m with their entries ±1
    psi_set = list(itertools.product([-1, 1], repeat=m))

    # This is the max m-height value, initially 0
    h_m_C = 0


    '''
    This nested function is used to solve the LP optimization of m-height for
    a fixed (a, b, X, psi)
    '''

    def solve_LP(a, b, X, psi):

        # Y is the set of remaining coordinates which are not in X, a, or b
        Y = list(set(indices) - set(X) - {a, b})

        # For psi index mapping, order of coordinates is: a, X, Y, b
        thou = [a] + sorted(list(X)) + Y + [b]

        # Objective funcion is: maximize psi[0] * <G[:,a], v> where v is a vector in R^k
        # Using -c here to reverse the objective, as linprog minimizes
        c = np.array([psi[0] * G[i, a] for i in range(k)])

        # The inequality constraints
        A_ub, b_ub = [], []

        # Following are the constraints for j in X
        for j in X:
            # first
            A_ub.append([psi[thou.index(j)] * G[i, j] - psi[0] * G[i, a] for i in range(k)])
            b_ub.append(0)
            # second
            A_ub.append([-psi[thou.index(j)] * G[i, j] for i in range(k)])
            b_ub.append(-1)

        # Following are the constraints for j in Y
        for j in Y:
            #The bound is: |<G[:,j], v>| <=1
            A_ub.append([G[i, j] for i in range(k)])
            b_ub.append(1)
            A_ub.append([-G[i, j] for i in range(k)])
            b_ub.append(1)

        # Equality constraint: <G[:,b], v> =1
        A_eq = [[G[i, b] for i in range(k)]]
        b_eq = [1]

        bounds = [(None, None)] * k
        res = linprog(-c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

        # If success then return the optimal value, else return 0
        return -res.fun if res.success else 0

    # Going over all the valid pairs (a, b)
    for a, b in itertools.permutations(indices, 2):
        # Going over all (m-1)-size subsets X excluding a, b
        for X in itertools.combinations(set(indices) - {a, b}, m - 1):
            # Going over all ±1 vectors of length m
            for psi in psi_set:
                z_val = solve_LP(a, b, X, psi)
                # Always storing the maximum height so far.
                h_m_C = max(h_m_C, z_val)
    return h_m_C

# List of 21 (n, k, m) combinations in the project scope
combinations = [
    (9, 4, 2), (9, 4, 3), (9, 4, 4), (9, 4, 5), (9, 5, 2), (9, 5, 3), (9, 5, 4),
    (9, 6, 2), (9, 6, 3), (10, 4, 2), (10, 4, 3), (10, 4, 4), (10, 4, 5), (10, 4, 6),
    (10, 5, 2), (10, 5, 3), (10, 5, 4), (10, 5, 5), (10, 6, 2), (10, 6, 3), (10, 6, 4)
]

# Generating and storing data
data = []
for i, (n, k, m) in enumerate(combinations, 1):
    # P part of the G matrix is generated and value of P ranges between [-100,100]
    P = np.random.uniform(-100, 100, size=(k, n - k))
    h_m_C = m_height_linprog_exact(P, n, k, m)
    data.append((n, k, m, h_m_C, P))
    print(f"[{i}/{len(combinations)}] Computed h_m(C) for (n={n}, k={k}, m={m}) -> {h_m_C}")

# Saving as a DataFrame
df = pd.DataFrame(data, columns=['n', 'k', 'm', 'h_m_C', 'P'])
df.to_pickle("m_height_dataset.pkl")
print("Dataset saved successfully!")


[1/21] Computed h_m(C) for (n=9, k=4, m=2) -> 144.90346620425922
[2/21] Computed h_m(C) for (n=9, k=4, m=3) -> 242.89075886905553
[3/21] Computed h_m(C) for (n=9, k=4, m=4) -> 1330.2166227995049
[4/21] Computed h_m(C) for (n=9, k=4, m=5) -> 4086.4253934219873
[5/21] Computed h_m(C) for (n=9, k=5, m=2) -> 321.2006862406292
[6/21] Computed h_m(C) for (n=9, k=5, m=3) -> 884.4806452981419
[7/21] Computed h_m(C) for (n=9, k=5, m=4) -> 10349.177498422843
[8/21] Computed h_m(C) for (n=9, k=6, m=2) -> 898.8751718194876
[9/21] Computed h_m(C) for (n=9, k=6, m=3) -> 39566.99255978265
[10/21] Computed h_m(C) for (n=10, k=4, m=2) -> 142.99082974010506
[11/21] Computed h_m(C) for (n=10, k=4, m=3) -> 158.20899105501928
[12/21] Computed h_m(C) for (n=10, k=4, m=4) -> 325.48136135587765
[13/21] Computed h_m(C) for (n=10, k=4, m=5) -> 3390.269358512461
[14/21] Computed h_m(C) for (n=10, k=4, m=6) -> 64123.66800799458
[15/21] Computed h_m(C) for (n=10, k=5, m=2) -> 178.21142924308538
[16/21] Computed h_

In [ ]:
import numpy as np
import pandas as pd
import itertools
from scipy.optimize import linprog
import os
import joblib

'''
This function is used to compute m-height and generate the dataset
The code runs 47619 time per combination to generate the dataset.
This was run on HPRC using a SLURM job.
'''

def m_height_linprog_exact(P, n, k, m):

    # Constructing generator matrix G = [I_k | P]
    G = np.hstack((np.eye(k), P))
    indices = list(range(n))

    # All the possible psi vectors of length m with their entries ±1
    psi_set = list(itertools.product([-1, 1], repeat=m))

    # This is the max m-height value, initially 0
    h_m_C = 0


    '''
    This nested function is used to solve the LP optimization of m-height for
    a fixed (a, b, X, psi)
    '''

    def solve_LP(a, b, X, psi):

        # Y is the set of remaining coordinates which are not in X, a, or b
        Y = list(set(indices) - set(X) - {a, b})

        # For psi index mapping, order of coordinates is: a, X, Y, b
        thou = [a] + sorted(list(X)) + Y + [b]

        # Objective funcion is: maximize psi[0] * <G[:,a], v> where v is a vector in R^k
        # Using -c here to reverse the objective, as linprog minimizes
        c = np.array([psi[0] * G[i, a] for i in range(k)])

        # The inequality constraints
        A_ub, b_ub = [], []

        # Following are the constraints for j in X
        for j in X:
            # first
            A_ub.append([psi[thou.index(j)] * G[i, j] - psi[0] * G[i, a] for i in range(k)])
            b_ub.append(0)
            # second
            A_ub.append([-psi[thou.index(j)] * G[i, j] for i in range(k)])
            b_ub.append(-1)

        # Following are the constraints for j in Y
        for j in Y:
            #The bound is: |<G[:,j], v>| <=1
            A_ub.append([G[i, j] for i in range(k)])
            b_ub.append(1)
            A_ub.append([-G[i, j] for i in range(k)])
            b_ub.append(1)

        # Equality constraint: <G[:,b], v> =1
        A_eq = [[G[i, b] for i in range(k)]]
        b_eq = [1]

        bounds = [(None, None)] * k
        res = linprog(-c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

        # If success then return the optimal value, else return
        return -res.fun if res.success else 0

    # Going over all the valid pairs (a, b)
    for a, b in itertools.permutations(indices, 2):
        # Going over all (m-1)-size subsets X excluding a, b
        for X in itertools.combinations(set(indices) - {a, b}, m - 1):
            # Going over all ±1 vectors of length m
            for psi in psi_set:
                z_val = solve_LP(a, b, X, psi)
                # Always storing the maximum height so far.
                h_m_C = max(h_m_C, z_val)
    return h_m_C

# List of 21 (n, k, m) combinations in the project scope
combinations = [
    (9, 4, 2), (9, 4, 3), (9, 4, 4), (9, 4, 5), (9, 5, 2), (9, 5, 3), (9, 5, 4),
    (9, 6, 2), (9, 6, 3), (10, 4, 2), (10, 4, 3), (10, 4, 4), (10, 4, 5), (10, 4, 6),
    (10, 5, 2), (10, 5, 3), (10, 5, 4), (10, 5, 5), (10, 6, 2), (10, 6, 3), (10, 6, 4)
]

# Generating and storing data
data = []

# Generating euqual number of samples for each combiation to get ~1M data
samples_per_combination = 47619

for i, (n, k, m) in enumerate(combinations, 1):
    for j in range(samples_per_combination):
        # P part of the G matrix is generated and value of P ranges between [-100,100]
        P = np.random.uniform(-100, 100, size=(k, n - k))
        h_m_C = m_height_linprog_exact(P, n, k, m)
        data.append((n, k, m, h_m_C, P))
        # print(f"[{i}/{len(combinations)}] Computed h_m(C) for (n={n}, k={k}, m={m}) -> {h_m_C}")

# Saving as a DataFrame in my directory
df = pd.DataFrame(data, columns=['n', 'k', 'm', 'h_m_C', 'P'])
output_dir = os.environ.get("SCRATCH") + "/dipanwita22rano/dlproject"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "m_height_dataset.pkl")
with open(output_path, 'wb') as f:
    joblib.dump(df, f)
print("Dataset saved successfully!")


In [ ]:
import joblib
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/m_height_dataset.pkl'
with open(file_path, 'rb') as f:
    df = joblib.load(f)

'''
  I had shuffled the dataset using this code, so that when I had made a single
  feed forward model, it sees random data, not similar data in sequence.
'''
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
output_path = '/content/drive/My Drive/m_height_dataset_.pkl'
with open(output_path, 'wb') as f:
    joblib.dump(df, f)
print(df.head(5))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
    n  k  m         h_m_C                                                  P
0  10  4  3    180.681884  [[0.41332842078512044, 70.03763016732799, 92.9...
1   9  6  3  29425.835225  [[-22.39439891822049, -45.59568035361643, 84.5...
2  10  4  4    590.143407  [[-98.04865191290197, 55.176558968450024, -47....
3   9  6  3   6838.797140  [[-72.01756032528372, -96.84222090558315, -97....
4   9  6  3   5219.964621  [[86.98714113704173, 62.11282763418615, -49.67...


Experimenting on LSTM + Bi-Encoder Model.
* As the order of rows in P does not hold semantic meaning, LSTMs can still try to extract relational patterns between rows that may influence the m-height through learned temporal dependencies.
* The Bidirectional LSTM acts as a bi-encoder by processing the matrix in both forward and backward directions. This should allow the model to encode contextual information from all surrounding rows, improving the representation of each row in relation to the others. This is especially helpful sine h_m_C may depend on global row interactions rather than isolated patterns.
* The metadata encoder should capture the structural information about the code, and its fusion with the LSTM output provides a joint representation of both the code matrix and its configuration.
This bi-encoder setup should help the model learn richer, more expressive features that generalize better across varying group structures.

But the experimenation results prove othrwise. The average test cost on the same dataset is worse than the transformer based model. This means the LSTM model stores history, which is not something needed for m-height problem.

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from tensorflow import keras
from keras import layers, regularizers

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/m_height_dataset.pkl'
with open(file_path, 'rb') as f:
    df = joblib.load(f)

df['k_val'] = df['P'].apply(lambda x: x.shape[0])
max_k = df['k_val'].max()
print(f"Maximum k (rows in P): {max_k}")

train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42)

def preprocess_group_lstm(group_df):
    X_list, y_list = [], []
    for _, row in group_df.iterrows():
        P = np.array(row['P']).astype(np.float32)
        n, k, m = row['n'], row['k'], row['m']
        meta = np.array([n, k, m], dtype=np.float32)

        padded_P = np.zeros((max_k, P.shape[1]), dtype=np.float32)
        padded_P[:P.shape[0], :] = P

        X_list.append((padded_P, meta))
        y_list.append(np.log2(row['h_m_C']))
    return X_list, np.array(y_list)


'''
  LSTM + Bi-Encoder Architecture
'''

def build_lstm_model(P_dim, meta_dim):
    P_input = keras.Input(shape=(max_k, P_dim), name='P_input')
    meta_input = keras.Input(shape=(meta_dim,), name='meta_input')

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(P_input)
    x = layers.GlobalAveragePooling1D()(x)

    combined = layers.Concatenate()([x, meta_input])
    combined = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(combined)
    combined = layers.Dropout(0.3)(combined)
    combined = layers.Dense(64, activation='relu')(combined)
    output = layers.Dense(1)(combined)

    model = keras.Model(inputs=[P_input, meta_input], outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model


os.makedirs('/content/models_lstm', exist_ok=True)

all_y_true_log2 = []
all_y_pred_log2 = []

unique_groups = sorted(train_df.groupby(['n', 'k', 'm']).groups.keys())

for (n, k, m) in unique_groups:
    train_group = train_df[(train_df['n'] == n) & (train_df['k'] == k) & (train_df['m'] == m)]
    val_group = val_df[(val_df['n'] == n) & (val_df['k'] == k) & (val_df['m'] == m)]
    test_group = test_df[(test_df['n'] == n) & (test_df['k'] == k) & (test_df['m'] == m)]

    if len(train_group) == 0 or len(val_group) == 0 or len(test_group) == 0:
        continue

    X_train_raw, y_train = preprocess_group_lstm(train_group)
    X_val_raw, y_val = preprocess_group_lstm(val_group)
    X_test_raw, y_test_log2 = preprocess_group_lstm(test_group)

    X_train_P = np.array([x[0] for x in X_train_raw])
    X_train_meta = np.array([x[1] for x in X_train_raw])
    X_val_P = np.array([x[0] for x in X_val_raw])
    X_val_meta = np.array([x[1] for x in X_val_raw])
    X_test_P = np.array([x[0] for x in X_test_raw])
    X_test_meta = np.array([x[1] for x in X_test_raw])

    model = build_lstm_model(P_dim=X_train_P.shape[2], meta_dim=3)
    model.fit(
        {'P_input': X_train_P, 'meta_input': X_train_meta},
        y_train,
        validation_data=({'P_input': X_val_P, 'meta_input': X_val_meta}, y_val),
        epochs=50,
        batch_size=32,
        verbose=0,
        callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
    )

    model_path = f'/content/models_lstm/model_n{n}_k{k}_m{m}.h5'
    model.save(model_path)

    y_pred_log2 = model.predict({'P_input': X_test_P, 'meta_input': X_test_meta}).flatten()
    y_pred = np.power(2, y_pred_log2)
    y_true = np.power(2, y_test_log2)

    y_pred = np.maximum(y_pred, 1.0)
    y_true = np.maximum(y_true, 1.0)

    all_y_pred_log2.extend(np.log2(y_pred))
    all_y_true_log2.extend(np.log2(y_true))


if all_y_true_log2 and all_y_pred_log2:
    all_y_true_log2 = np.array(all_y_true_log2)
    all_y_pred_log2 = np.array(all_y_pred_log2)

    rmsle = np.sqrt(mean_squared_log_error(all_y_true_log2, all_y_pred_log2))
    test_cost = np.mean(np.square(all_y_true_log2 - all_y_pred_log2))

    print(f"\nOverall RMSLE (log₂): {rmsle:.4f}")
    print(f"Average test cost (log₂ squared error): {test_cost:.4f}")
else:
    print("No valid predictions to evaluate.")


Mounted at /content/drive
Maximum k (rows in P): 6


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


...

201/201 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


262/262 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


262/262 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

Overall RMSLE (log₂): 0.1291
Average test cost (log₂ squared error): 2.3536


Experimenting on a Set Transformer Model:
* It was chosen to better align with the nature of the generator matrix, which can be viewed as a set of row vectors.
* Unlike LSTMs, which impose an order on the input, Set Transformers are permutation-invariant. (To experiment whether row ordering gives better results or without ordering.)
* Each row of P is concatenated with metadata (n, k, m), which should allow the model to consider not just individual row characteristics, but also how each row cotributes in the context of the full code specification.
* The multi-headed self-attention blocks should allow the model to learn pairwise interactions between rows which might enable it to model higher-order dependencies and relations that might contribute to the m-height.

But, this experimentation showed to have the worst results. This means we need to have a sense of row-order structure maintenance in the architecture.

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from tensorflow import keras
from keras import layers, regularizers
import tensorflow as tf

drive.mount('/content/drive')
file_path = '/content/drive/My Drive/m_height_dataset.pkl'
with open(file_path, 'rb') as f:
    df = joblib.load(f)

df['k_val'] = df['P'].apply(lambda x: x.shape[0])
max_k = df['k_val'].max()
print(f"Maximum k (rows in P): {max_k}")

train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42)

# preprocessimg to be compatible with the set format
def preprocess_group(group_df):
    X_list, y_list = [], []
    for _, row in group_df.iterrows():
        P = np.array(row['P']).astype(np.float32)
        n, k, m = row['n'], row['k'], row['m']
        meta = np.array([n, k, m], dtype=np.float32)

        row_features = []
        for i in range(P.shape[0]):
            features = np.concatenate([P[i], meta])
            row_features.append(features)

        # Zero padding
        while len(row_features) < max_k:
            row_features.append(np.zeros_like(row_features[0]))

        X = np.array(row_features)
        X_list.append(X)
        y_list.append(np.log2(row['h_m_C']))
    return np.array(X_list), np.array(y_list)

'''
  Set Transformer Architecture
'''

def build_set_transformer(input_shape):
    inputs = keras.Input(shape=input_shape)

    # 2 perumation invariant self attention blocks
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
    x = layers.LayerNormalization()(x)
    x = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
    x = layers.LayerNormalization()(x)

    x = layers.GlobalAveragePooling1D()(x)

    # MLP head
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    outputs = layers.Dense(1)(x)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

os.makedirs('/content/models', exist_ok=True)
all_y_true_log2, all_y_pred_log2 = [], []

unique_groups = sorted(train_df.groupby(['n', 'k', 'm']).groups.keys())

for (n, k, m) in unique_groups:
    train_group = train_df[(train_df['n'] == n) & (train_df['k'] == k) & (train_df['m'] == m)]
    val_group = val_df[(val_df['n'] == n) & (val_df['k'] == k) & (val_df['m'] == m)]
    test_group = test_df[(test_df['n'] == n) & (test_df['k'] == k) & (test_df['m'] == m)]

    if len(train_group) == 0 or len(val_group) == 0 or len(test_group) == 0:
        continue

    X_train, y_train = preprocess_group(train_group)
    X_val, y_val = preprocess_group(val_group)
    X_test, y_test_log2 = preprocess_group(test_group)

    model = build_set_transformer(input_shape=X_train.shape[1:])
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        verbose=0,
        callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
    )

    model_path = f'/content/models/model_n{n}_k{k}_m{m}.h5'
    model.save(model_path)

    y_pred_log2 = model.predict(X_test).flatten()
    y_pred = np.power(2, y_pred_log2)
    y_true = np.power(2, y_test_log2)

    y_pred = np.maximum(y_pred, 1.0)
    y_true = np.maximum(y_true, 1.0)

    all_y_pred_log2.extend(np.log2(y_pred))
    all_y_true_log2.extend(np.log2(y_true))


if all_y_true_log2 and all_y_pred_log2:
    all_y_true_log2 = np.array(all_y_true_log2)
    all_y_pred_log2 = np.array(all_y_pred_log2)

    rmsle = np.sqrt(mean_squared_log_error(all_y_true_log2, all_y_pred_log2))
    test_cost = np.mean(np.square(all_y_true_log2 - all_y_pred_log2))

    print(f"\nFinal RMSLE (log₂): {rmsle:.4f}")
    print(f"Average test cost (log₂ squared error): {test_cost:.4f}")
else:
    print("No valid predictions to evaluate.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Maximum k (rows in P): 6


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


...

201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


262/262 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


262/262 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

Final RMSLE (log₂): 0.1181
Average test cost (log₂ squared error): 2.0712


This is the final model I came up with, with the lowest RMSLE amongst all other methods that i tried. This set of 21 models use transformer models, with one-hot encoding of row indices.

Other models that I had tried in Project 1 were:
* Single general model of feed forward architecture. It consisted of 4 layers, regularization, and an extensive hyperparameter tuning was done on it. Test RMSLE was way higher than validation.
* 21 models for each combination of (n, k, m) but all feed forward. Even though this reduced the RMSLE, but it predicted same values for a combination of (n, k, m). Basically this model only learnt a value for each combination, which would minimize the loss.
* As P is a matrix, I thought of using CNN, to have a 2D input. But this proved to be the most adverserial, having RMSLE even as high as 57. Thus, it proved that for m-height calculation, there is no spatial relationship between the matrix cell values.

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from tensorflow import keras
from keras import layers, regularizers
import tensorflow as tf

'''
  The dataset was saved to my google drive, so that I do not need to repeatedly
  upload it to my Colab runtime.
'''
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/m_height_dataset.pkl'
with open(file_path, 'rb') as f:
    df = joblib.load(f)

'''
  Computing the maximum number of rows in P for Positional Encoding.
  It is used to fix the size of one-hot encoding
  This gets the maximum value seen across all samples,
  so it must be 6 according to the project scope, else error in dataset generation.
'''
df['k_val'] = df['P'].apply(lambda x: x.shape[0])
max_k = df['k_val'].max()
print(f"Maximum k (rows in P): {max_k}")

'''
  Splitting the dataset here:
  Test = 15% of df
  Train = 85% * (85%) of df
  Validation = 15% * (85%) of df
'''
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42)

'''
  The following preprocessing function:
  Adds metadata (n, k, m) and positional encoding to each row of P
  I tested without adding metadata, as anyways making individual models for each combination
  But the results were adversarial with poor RMSLE values

  Output shape: (k, feature_dim) where feature_dim = (columns in P + 3 metadata + max_k one-hot)
  Target: log(h_m_C)
  I tested the target without its log value, that is, the actual m-height.
  But the RMSLE is better when the model is trained on log(m-height)
'''
def preprocess_group(group_df):
    X_list, y_list = [], []
    for _, row in group_df.iterrows():
        # P has the shape (k, n-k)
        P = np.array(row['P']).astype(np.float32)
        n, k, m = row['n'], row['k'], row['m']
        meta = np.array([n, k, m], dtype=np.float32)

        row_features = []
        for i in range(P.shape[0]):
            pos_encoding = np.zeros(max_k)
            # One-hot encoding the row index here
            pos_encoding[i] = 1.0
            features = np.concatenate([P[i], meta, pos_encoding])
            row_features.append(features)

        X = np.array(row_features)
        X_list.append(X)
        # Log scaling the target here
        y_list.append(np.log(row['h_m_C']))
    return np.array(X_list), np.array(y_list)

'''
  After much experimentation, developed a transformer based model.
  This stems from the idea that m-height may depend on interactions between different rows.
  Thus a transformer can model pairwise and gobal dependencies via self-attention.

  Using transformer is better than fully connected layers alone because,
  MLPs treat all rows flatly and independently,
  CNNs assume local patterns, which is not present in a mathematical concept,
  Thus transformers can learn attention weights dynamically across all rows.
'''
def build_transformer_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Normalization
    x = layers.LayerNormalization()(inputs)
    # Self-attention
    x = layers.MultiHeadAttention(num_heads=2, key_dim=8)(x, x)
    # For variable-length inputs
    x = layers.GlobalAveragePooling1D()(x)

    # Dense layer with L2 regularization
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    # Finally predict a single log(h_m_C) value
    outputs = layers.Dense(1)(x)

    model = keras.Model(inputs, outputs)
    # MSE loss is being used in log-scale
    model.compile(optimizer='adam', loss='mse')
    return model

# Training, saving and evaluating model for each combination of (n, k, m) groups
results = []
unique_groups = sorted(train_df.groupby(['n', 'k', 'm']).groups.keys())

os.makedirs('/content/models', exist_ok=True)

for (n, k, m) in unique_groups:
    print(f"\nTraining model for (n={n}, k={k}, m={m})")

    # Filtering group-specific data from each dataset split
    train_group = train_df[(train_df['n'] == n) & (train_df['k'] == k) & (train_df['m'] == m)]
    val_group = val_df[(val_df['n'] == n) & (val_df['k'] == k) & (val_df['m'] == m)]
    test_group = test_df[(test_df['n'] == n) & (test_df['k'] == k) & (test_df['m'] == m)]

    # Skipping groups which do not have even a single data, though this condition should never be reached
    if len(train_group) == 0 or len(val_group) == 0 or len(test_group) == 0:
        print("Skipping due to insufficient data.")
        continue

    # Applying preprocessing on each split
    X_train, y_train = preprocess_group(train_group)
    X_val, y_val = preprocess_group(val_group)
    X_test, y_test_log = preprocess_group(test_group)

    # Training and model building
    # Keeping epochs 30, as plotting training and validation loss curved showed not much change in loss after 27
    model = build_transformer_model(input_shape=X_train.shape[1:])
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        verbose=0,
        callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
    )

    model_path = f'/content/models/model_n{n}_k{k}_m{m}.h5'
    model.save(model_path)
    print(f"Model saved to {model_path}")

    # Prediction on the test set
    # Log base e value is predicted, which is converted to actual value
    y_pred_log = model.predict(X_test).flatten()
    y_pred = np.exp(y_pred_log)
    y_true = np.exp(y_test_log)

    # Clipping predictions to ensure valid domain for log2
    y_pred = np.maximum(y_pred, 1.0)
    y_true = np.maximum(y_true, 1.0)

    # Computing RMSLE in log base 2 space
    # This is done as the test cost function in the project scope suggests so
    y_pred_log2 = np.log2(y_pred)
    y_true_log2 = np.log2(y_true)
    rmsle = np.sqrt(mean_squared_log_error(y_true_log2, y_pred_log2))
    results.append(((n, k, m), rmsle))
    print(f"RMSLE (log base 2): {rmsle:.4f}")

print("\n=== Summary of Results ===")
for (n, k, m), rmsle in results:
    print(f"Group (n={n}, k={k}, m={m}): RMSLE (log base 2) = {rmsle:.4f}")


Mounted at /content/drive
Maximum k (rows in P): 6

Training model for (n=9, k=4, m=2)


Model saved to /content/models/model_n9_k4_m2.h5
195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
RMSLE (log base 2): 0.0539

Training model for (n=9, k=4, m=3)


Model saved to /content/models/model_n9_k4_m3.h5
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
RMSLE (log base 2): 0.0522

Training model for (n=9, k=4, m=4)


Model saved to /content/models/model_n9_k4_m4.h5
195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
RMSLE (log base 2): 0.0833

Training model for (n=9, k=4, m=5)


Model saved to /content/models/model_n9_k4_m5.h5
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
RMSLE (log base 2): 0.1157

Training model for (n=9, k=5, m=2)


...

Model saved to /content/models/model_n10_k5_m4.h5
201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
RMSLE (log base 2): 0.0790

Training model for (n=10, k=5, m=5)


Model saved to /content/models/model_n10_k5_m5.h5
195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
RMSLE (log base 2): 0.1049

Training model for (n=10, k=6, m=2)


Model saved to /content/models/model_n10_k6_m2.h5
262/262 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
RMSLE (log base 2): 0.0500

Training model for (n=10, k=6, m=3)


Model saved to /content/models/model_n10_k6_m3.h5
262/262 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
RMSLE (log base 2): 0.0767

Training model for (n=10, k=6, m=4)


Model saved to /content/models/model_n10_k6_m4.h5
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
RMSLE (log base 2): 0.1030

=== Summary of Results ===
Group (n=9, k=4, m=2): RMSLE (log base 2) = 0.0539
Group (n=9, k=4, m=3): RMSLE (log base 2) = 0.0522
Group (n=9, k=4, m=4): RMSLE (log base 2) = 0.0833
Group (n=9, k=4, m=5): RMSLE (log base 2) = 0.1157
Group (n=9, k=5, m=2): RMSLE (log base 2) = 0.0503
Group (n=9, k=5, m=3): RMSLE (log base 2) = 0.0809
Group (n=9, k=5, m=4): RMSLE (log base 2) = 0.1095
Group (n=9, k=6, m=2): RMSLE (log base 2) = 0.0716
Group (n=9, k=6, m=3): RMSLE (log base 2) = 0.1149
Group (n=10, k=4, m=2): RMSLE (log base 2) = 0.1386
Group (n=10, k=4, m=3): RMSLE (log base 2) = 0.0377
Group (n=10, k=4, m=4): RMSLE (log base 2) = 0.0554
Group (n=10, k=4, m=5): RMSLE (log base 2) = 0.0825
Group (n=10, k=4, m=6): RMSLE (log base 2) = 0.1109
Group (n=10, k=5, m=2): RMSLE (log base 2) = 0.0384
Group (n=10, k=5, m=3): RMSLE (log base 2) = 0.0582
Group (n=10, k=5, m=4): RMSLE (

Reasons I think this transformer based model works better tha any other because:
* Enriching ecah row with Metadata (n, k, m) influences the m-height.
* Positional encoding helps the model differentiate between rows and potentially learn row-wise importance or order effects.
* The model seems to behave a little contextualized, position-aware features, and improves generalization and interpretability.
* Using traditional MLPs would treat the input as flat and ignore row-level structure.
* CNNs ae treating it like a grid, focusing on local row-wise patterns but here m-height is a global algebraic propert, not a spatial one.
* Self attention is allowing the model to learn global dependencies, which is critical since any row could influence the m-height based on its interaction with any other.
* Training individual models per (n, k, m) here avoids forcing a single model to learn a complex, multi-distribution space.. This in turn results in smaller, specalized models that overfit less and generalize better within each group.
* from experiments, training on log(h_m_C) produces better results. This maybe due to it reducing skew and improving learning, especially since m-height values can vary exponentially.
* L2 regularization in the dense layers and early stopping controls overfitting and prevents the model from memorizing small training sets in low-data regimes per group.

Next I generated another dataset of 100k rows, to further check how the model is performing on completely unseen data, and data that has been generated in a different session. This dataset was saved in my drive with the name 'm_height_dataset_1k.pkl'

In [ ]:
import os
import joblib
import numpy as np
from zipfile import ZipFile
from google.colab import drive
from tensorflow import keras
from sklearn.metrics import mean_squared_log_error

'''
  Usual preprocessing done before too while training
'''
drive.mount('/content/drive')

test_file_path = '/content/drive/My Drive/m_height_100k.pkl'
with open(test_file_path, 'rb') as f:
    test_df = joblib.load(f)


# Fixing max_k based on analysis during training
max_k = 6


def preprocess_group(group_df):
    X_list, y_list = [], []
    for _, row in group_df.iterrows():
        P = np.array(row['P']).astype(np.float32)
        n, k, m = row['n'], row['k'], row['m']
        meta = np.array([n, k, m], dtype=np.float32)

        row_features = []
        for i in range(P.shape[0]):
            pos_encoding = np.zeros(max_k)
            pos_encoding[i] = 1.0
            features = np.concatenate([P[i], meta, pos_encoding])
            row_features.append(features)

        X = np.array(row_features)
        X_list.append(X)
        y_list.append(np.log(row['h_m_C']))
    return np.array(X_list), np.array(y_list)

'''
  The final model was saved with the name 'models3.zip'.
  That is now uploaded to runtime and unzipped.
  The test data is first grouped into bins of each combinations
  of (n, k, m), and then the pertaining model is invoked.
'''
model_zip_path = 'models3.zip'
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

with ZipFile(model_zip_path, 'r') as zip_ref:
    zip_ref.extractall(model_dir)

# These variables keep track of the evaluation metrics, globally across all models
all_preds_log2 = []
all_trues_log2 = []

custom_objects = {'mse': keras.losses.MeanSquaredError()}
# Splitting data into groups
unique_groups = sorted(test_df.groupby(['n', 'k', 'm']).groups.keys())

for (n, k, m) in unique_groups:
    model_path = os.path.join(model_dir, f"model_n{n}_k{k}_m{m}.h5")
    #This condition should not be reached, but an error check
    if not os.path.exists(model_path):
        continue

    group_df = test_df[(test_df['n'] == n) & (test_df['k'] == k) & (test_df['m'] == m)]
    if len(group_df) == 0:
        continue

    X_test, y_test_log = preprocess_group(group_df)
    model = keras.models.load_model(model_path, custom_objects=custom_objects)

    y_pred_log = model.predict(X_test).flatten()
    y_pred = np.exp(y_pred_log)
    y_true = np.exp(y_test_log)

    y_pred = np.maximum(y_pred, 1.0)
    y_true = np.maximum(y_true, 1.0)

    all_preds_log2.extend(np.log2(y_pred))
    all_trues_log2.extend(np.log2(y_true))

    # To check the results, printing some actual vs predicted m-heights
    print(f"\n--- Predictions for Group (n={n}, k={k}, m={m}) ---")
    num_samples = min(len(y_true), 5)
    for i in range(num_samples):
        print(f"Sample {i+1}: Actual = {y_true[i]:.2f}, Predicted = {y_pred[i]:.2f}")

# Calculating all the metrics globally
if all_preds_log2:
    all_preds_log2 = np.array(all_preds_log2)
    all_trues_log2 = np.array(all_trues_log2)

    global_rmsle = np.sqrt(mean_squared_log_error(all_trues_log2, all_preds_log2))
    avg_cost = np.mean((all_trues_log2 - all_preds_log2) ** 2)

    print("\n=== Global Test Evaluation ===")
    print(f"Total Samples: {len(all_preds_log2)}")
    print(f"Global Average Cost σ: {avg_cost:.4f}")
    print(f"Global RMSLE (log base 2): {global_rmsle:.4f}")
else:
    print("No valid predictions found.")


Mounted at /content/drive


128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

--- Predictions for Group (n=9, k=4, m=2) ---
Sample 1: Actual = 194.03, Predicted = 170.31
Sample 2: Actual = 175.25, Predicted = 132.05
Sample 3: Actual = 152.03, Predicted = 157.95
Sample 4: Actual = 176.14, Predicted = 159.20
Sample 5: Actual = 183.21, Predicted = 152.08


131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

--- Predictions for Group (n=9, k=4, m=3) ---
Sample 1: Actual = 210.28, Predicted = 270.19
Sample 2: Actual = 177.91, Predicted = 270.64
Sample 3: Actual = 220.43, Predicted = 260.80
Sample 4: Actual = 252.63, Predicted = 289.72
Sample 5: Actual = 215.02, Predicted = 253.29


128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

--- Predictions for Group (n=9, k=4, m=4) ---
Sample 1: Actual = 3134.83, Predicted = 1111.45
Sample 2: Actual = 1434.17, Predicted = 933.10
Sample 3: Actual = 1550.49, Predicted = 1034.68
Sample 4: Actual = 1109.48, Predicted = 976.30
Sample 5: Actual = 684.32, Predicted = 865.91


131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

--- Predictions for Group (n=9, k=4, m=5) ---
Sample 1: Actual = 152304.11, Predicted = 21039.54
Sample 2: Actual = 17305.33, Predicted = 25083.43
Sample 3: Actual = 8202.88, Predicted = 20897.84
Sample 4: Actual = 32600.17, Predicted = 24789.27
Sample 5: Actual = 20170.64, Predicted = 23110.59


...

133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

--- Predictions for Group (n=10, k=5, m=4) ---
Sample 1: Actual = 3283.15, Predicted = 2201.54
Sample 2: Actual = 1513.29, Predicted = 2028.42
Sample 3: Actual = 3004.40, Predicted = 2027.26
Sample 4: Actual = 5171.45, Predicted = 1939.58
Sample 5: Actual = 1494.52, Predicted = 2051.57


132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

--- Predictions for Group (n=10, k=5, m=5) ---
Sample 1: Actual = 177047.48, Predicted = 70163.84
Sample 2: Actual = 246821.81, Predicted = 70372.19
Sample 3: Actual = 23983.40, Predicted = 69214.73
Sample 4: Actual = 29059.81, Predicted = 72192.05
Sample 5: Actual = 41024.03, Predicted = 72784.98


176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

--- Predictions for Group (n=10, k=6, m=2) ---
Sample 1: Actual = 738.55, Predicted = 436.10
Sample 2: Actual = 413.91, Predicted = 449.45
Sample 3: Actual = 285.07, Predicted = 405.66
Sample 4: Actual = 1027.39, Predicted = 403.71
Sample 5: Actual = 874.60, Predicted = 492.77


171/171 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

--- Predictions for Group (n=10, k=6, m=3) ---
Sample 1: Actual = 1619.49, Predicted = 1850.11
Sample 2: Actual = 1884.39, Predicted = 1728.86
Sample 3: Actual = 1319.60, Predicted = 1773.84
Sample 4: Actual = 1010.55, Predicted = 1592.58
Sample 5: Actual = 4016.91, Predicted = 1752.02


175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

--- Predictions for Group (n=10, k=6, m=4) ---
Sample 1: Actual = 137656.18, Predicted = 68381.42
Sample 2: Actual = 40116.68, Predicted = 65603.41
Sample 3: Actual = 31498.34, Predicted = 69660.20
Sample 4: Actual = 485405.97, Predicted = 67977.70
Sample 5: Actual = 246626.37, Predicted = 68896.18

=== Global Test Evaluation ===
Total Samples: 100000
Global Average Cost σ: 1.4477
Global RMSLE (log base 2): 0.0853


The next cell runs the evaluation on the test set. As said in the project scope, upload the models directory shared, named, '436000715_models.zip'.
The function call has been done. Just update the dictionary 'inputs', to your evaluation dictionary. The format has been followed as in the template. Next, the cell returns a dictionary as output, and prints it.

After inserting your input in the placeholder, and uploading the zip file, run the following cell.

In [ ]:
import os
import joblib
import numpy as np
from zipfile import ZipFile
from google.colab import drive
from tensorflow import keras
from sklearn.metrics import mean_squared_log_error

def predict_m_heights(inputs, model_dir, max_k=6):
    outputs = {}
    custom_objects = {'mse': keras.losses.MeanSquaredError()}

    for key, matrices in inputs.items():
        # Extracting n, k, m from the dictionary
        n, k, m = eval(key)
        model_path = os.path.join(model_dir, f"model_n{n}_k{k}_m{m}.h5")

        # Skipping, in case model is not found
        if not os.path.exists(model_path):
            print(f"Model for group (n={n}, k={k}, m={m}) not found.")
            continue

        # Loading the model here
        model = keras.models.load_model(model_path, custom_objects=custom_objects)
        preds = []

        # Preprocessing the P_list
        for P in matrices:
            P = np.array(P).astype(np.float32)
            meta = np.array([n, k, m], dtype=np.float32)

            row_features = []
            for i in range(P.shape[0]):
                pos_encoding = np.zeros(max_k)
                pos_encoding[i] = 1.0
                features = np.concatenate([P[i], meta, pos_encoding])
                row_features.append(features)

            X = np.array([row_features])
            y_pred_log = model.predict(X, verbose=0).flatten()[0]
            y_pred = np.exp(y_pred_log)
            y_pred = max(1.0, y_pred)
            preds.append(y_pred)

        outputs[key] = preds

    return outputs


# Sample input is shown, modify the input with test set here
'''
inputs = {
    '[9,6,3]': [
        np.array([
            [ 0.4759809,  0.9938236, 0.819425 ],
            [-0.8960798, -0.7442706, 0.3345122],
            [ 0.4759809,  0.9938236, 0.819425 ],
            [-0.8960798, -0.7442706, 0.3345122],
            [ 0.4759809,  0.9938236, 0.819425 ],
            [-0.8960798, -0.7442706, 0.3345122],
        ]),
        # Add more P matrices here if needed

    ],
}
'''
# Insert here
inputs = {

}

model_zip_path = '436000715_models.zip'
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

with ZipFile(model_zip_path, 'r') as zip_ref:
    zip_ref.extractall(model_dir)

# Calling the required function here
outputs = predict_m_heights(inputs, model_dir, max_k=6)

print("\n=== Outputs ===")
for key in outputs:
    preds = [float(x) for x in outputs[key]]
    print(f"{key} => {preds}")


=== Outputs ===
[9,6,3] => [4725.53173828125]
